### Урок 12. Домашняя работа

**Задача высокого уровня** В реализацию функции `gradient` добавьте параметр $\lambda$, чтобы получить регуляризованный градиентный спуск

Формула поменяется следующим образом:
$$
\left\{
\begin{array}{cc}
\frac{\partial L}{\partial w_0} = \frac{2}{n}\cdot(-1)\cdot \sum_{i=1}^{n} (1\cdot \left(y_i - \sum_{j=1}^{m}w_jx_j^i\right) + \lambda\cdot 2\cdot w_0)&\\
\frac{\partial L}{\partial w_k} = \frac{2}{n}\cdot(-1)\cdot \sum_{i=1}^{n} (x_k^i \cdot\left(y_i - \sum_{j=1}^{m}w_jx_j^i\right) + \lambda\cdot 2\cdot w_k)& k\neq 0 \\
\end{array}
\right.
$$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import distance

from sklearn.linear_model import SGDRegressor,LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('non_linear.csv', sep=',')
data = data[(data.x_train > 1) & (data.x_train < 5)].copy()
data.head()

,x_train,y_train
5,1.182421,1.860341
6,1.251605,1.878928
7,1.270474,2.430015
8,1.402553,2.327856
9,1.427711,2.203649


## **обычный градиентный спуск**
$$
\left\{
\begin{array}{cc}
\frac{\partial L}{\partial w_0} = \frac{2}{n}\cdot(-1)\cdot \sum_{i=1}^{n} (1\cdot \left(y_i - \sum_{j=1}^{m}w_jx_j^i\right))&\\
\frac{\partial L}{\partial w_k} = \frac{2}{n}\cdot(-1)\cdot \sum_{i=1}^{n} (x_k^i \cdot\left(y_i - \sum_{j=1}^{m}w_jx_j^i\right))& k\neq 0 \\
\end{array}
\right.
$$


In [29]:
def gradient_descent (X, y, eta = 0.008, epsilon = 0.001, fit_intercept = True):
    "Функция обычного градиетного спуска"
           
    step = 0
    weight_evolution = np.inf
    # количество обучающих примеров в выборке
    n = X.shape[0]
    # количество признаков
    m = X.shape[1] 

    if fit_intercept == True:
        # добавляем тривиальный признак w_0, столбец из единиц
        X = np.hstack([np.ones(n).reshape(-1,1), X])
        # количество признаков
        m = X.shape[1]
        
    # инициализируем рандомом веса
    w=[]
    w = np.random.random(m).reshape(1, -1)
        
    while weight_evolution > epsilon:  # повторяем до сходимости вектора весов
        step += 1

        # считаем прогноз
        y_hat = X.dot(w.T)
        # вычисляем ошибку прогноза
        error = y - y_hat
        # вычисляем градиент дальше pointwise перемножение - умножаем каждую из координат на ошибку
        grad = np.multiply(X, error).sum(axis=0)*(-1.0)*2.0 / n
        # делаем шаг градиентного спуска
        w_next = w - eta * grad
        # проверяем условие сходимости
        weight_evolution = distance.euclidean(w, w_next)
        w = w_next
    
        if step % 100 ==0:
            print("step %s |w-w_next|=%.5f, grad=%s" % (step, weight_evolution, grad))
    
        if step > 5000: 
            break
    return w
    

In [42]:
# трансформируем плоский массив X в вектор-столбец
X = data['x_train'].values.reshape(-1, 1)
# n = X.shape[0]
# добавляем тривиальный признак w_0, столбец из единиц
# X = np.hstack([np.ones(n).reshape(-1,1), X])
y = data['y_train'].values.reshape(-1, 1)

a = True # fit_intercept
W = gradient_descent (X, y, eta=0.01, fit_intercept = a)

step 100 |w-w_next|=0.00511, grad=[-0.49124676  0.14191142]
step 200 |w-w_next|=0.00401, grad=[-0.38501243  0.11122243]
step 300 |w-w_next|=0.00314, grad=[-0.30175175  0.08717008]
step 400 |w-w_next|=0.00246, grad=[-0.23649657  0.06831916]
step 500 |w-w_next|=0.00193, grad=[-0.18535312  0.05354483]
step 600 |w-w_next|=0.00151, grad=[-0.14526967  0.04196552]
step 700 |w-w_next|=0.00119, grad=[-0.11385445  0.03289029]


In [43]:
fit_intercept = a
if fit_intercept == True:
    X = data['x_train'].values.reshape(-1, 1)
    n = X.shape[0]
    X = np.hstack([np.ones(n).reshape(-1,1), X])
else:
    X = data['x_train'].values.reshape(-1, 1)
    
y_pred = X.dot(W.T)
mean_squared_error(y, y_pred)

0.12241906910716566

In [44]:
# проверка - значение mse не совпадает со значением, полученным, когда реализую градиентный спуск вручную
sgd_regressor = SGDRegressor(learning_rate = 'constant', eta0 = 0.01, fit_intercept = a, random_state = 42)
reg = SGDRegressor().fit(X, y)
y_pred = reg.predict(X)
mean_squared_error(y, y_pred)

C:\Python\Anaconda\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.17953174261309246

## **градиентный спуск с регуляризацией**
$$
\left\{
\begin{array}{cc}
\frac{\partial L}{\partial w_0} = \frac{2}{n}\cdot(-1)\cdot \sum_{i=1}^{n} (1\cdot \left(y_i - \sum_{j=1}^{m}w_jx_j^i\right) + \lambda\cdot 2\cdot w_0)&\\
\frac{\partial L}{\partial w_k} = \frac{2}{n}\cdot(-1)\cdot \sum_{i=1}^{n} (x_k^i \cdot\left(y_i - \sum_{j=1}^{m}w_jx_j^i\right) + \lambda\cdot 2\cdot w_k)& k\neq 0 \\
\end{array}
\right.
$$

In [45]:
def gradient_descent_reg (X, y, eta = 0.008, epsilon = 0.001, l = 0.01, fit_intercept = True):
    "Функция градиетного спуска с регуляризацией L2"
    
    step = 0
    weight_evolution = np.inf
    # количество обучающих примеров в выборке
    n = X.shape[0]
    # количество признаков
    m = X.shape[1]

    if fit_intercept == True:
        # добавляем тривиальный признак w_0, столбец из единиц
        X = np.hstack([np.ones(n).reshape(-1,1), X])
        # количество признаков
        m = X.shape[1]
        
    # инициализируем рандомом веса
    w=[]
    w = np.random.random(m).reshape(1, -1)
    
    while weight_evolution > epsilon:  # повторяем до сходимости вектора весов
        step += 1

        # считаем прогноз
        y_hat = X.dot(w.T)
        # вычисляем ошибку прогноза
        error = y - y_hat
        # вычисляем градиент дальше pointwise перемножение - умножаем каждую из координат на ошибку
        grad = (np.multiply(X, error) + 2 * l * w).sum(axis=0)*(-1.0)*2.0 / n 
        # делаем шаг градиентного спуска
        w_next = w - eta * grad
        # проверяем условие сходимости
        weight_evolution = distance.euclidean(w, w_next)
        w = w_next
    
        if step % 100 ==0:
            print("step %s |w-w_next|=%.5f, grad=%s" % (step, weight_evolution, grad))
    
        if step > 5000: 
            break
    return w

In [47]:
# трансформируем плоский массив X в вектор-столбец
X = data['x_train'].values.reshape(-1, 1)
# n = X.shape[0]
# добавляем тривиальный признак w_0, столбец из единиц
# X = np.hstack([np.ones(n).reshape(-1,1), X])
y = data['y_train'].values.reshape(-1, 1)

a = True # fit_intercept
W = gradient_descent (X, y, eta=0.01, fit_intercept = a)

step 100 |w-w_next|=0.00443, grad=[-0.42547571  0.12291147]
step 200 |w-w_next|=0.00347, grad=[-0.33346466  0.09633131]
step 300 |w-w_next|=0.00272, grad=[-0.26135143  0.07549923]
step 400 |w-w_next|=0.00213, grad=[-0.204833    0.05917218]
step 500 |w-w_next|=0.00167, grad=[-0.16053694  0.04637593]
step 600 |w-w_next|=0.00131, grad=[-0.1258201   0.03634692]
step 700 |w-w_next|=0.00103, grad=[-0.09861094  0.02848674]


In [48]:
fit_intercept = a
if fit_intercept == True:
    X = data['x_train'].values.reshape(-1, 1)
    n = X.shape[0]
    X = np.hstack([np.ones(n).reshape(-1,1), X])
else:
    X = data['x_train'].values.reshape(-1, 1)
    
y_pred = X.dot(W.T)
mean_squared_error(y, y_pred)

0.1224201776980459

In [49]:
# проверка - значение mse не совпадает со значением, полученным, когда реализую градиентный спуск вручную
sgd_regressor = SGDRegressor(penalty = 'l2', alpha = 0.01, learning_rate = 'constant', eta0 = 0.01,\
                             fit_intercept = a, random_state = 42)
reg = SGDRegressor().fit(X, y)
y_pred = reg.predict(X)
mean_squared_error(y, y_pred)

C:\Python\Anaconda\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.1804080936393841

В этом модуле мы узнали, как  обучать линейную регрессию, не "упираясь" в аппаратные ресурсы: использовать градиентный спуск.
Мы узнали, как детектировать переобучение модели и закрепили свои знания на примере полиномиальной регрессии и выяснили, как увеличить качество решения с помощью механизма регуляризации. Познакомились с двумя видами регуляризации -  Ridge и Lasso.